In [37]:
import numpy as np
import statsmodels.tsa.stattools as ts 
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import pandas as pd
import wrds
import statsmodels.api as sm

In [38]:
# Login
import wrds
db = wrds.Connection(wrds_username="hanyuzhang")
# password is Timeseries2020!

Enter your WRDS username [liyf4]:hanyuzhang
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [39]:
# Preprocess Data
import pandas as pd
sym_df = pd.read_csv("Ticker.csv", header=0)
sym_list = sym_df["Ticker"].tolist()
sym_permno_list = list()
for symbol in sym_list:
    if len(symbol) == 0:
        continue
    result = db.raw_sql(f"""select permno, htsymbol 
                           from crsp.dsfhdr 
                           where htsymbol = '{symbol}'""")
    try:
        sym_permno_list.append((result.iloc[0]['permno'], result.iloc[0]['htsymbol']))
    except:
        print(f"wrds doesn't have data for {symbol} right now, skip...")

FileNotFoundError: [Errno 2] No such file or directory: 'Ticker.csv'

In [4]:
from collections import defaultdict
timeseries_bank = defaultdict(list)
for permno, symbol in sym_permno_list:
    result = db.raw_sql(f"""select date, prc, cfacpr
                            from crsp.dsf
                            where permno = {permno} and date > '2010-01-01'
                         """)
    ts = {date:prc for date, prc in zip(result["date"].tolist(), (result["prc"]/result["cfacpr"]).tolist())}
    timeseries_bank[symbol] = ts  

In [5]:

# Build pairs
sym_pairs = list()
sym_pairs_ts = dict()
valid_sym_list = list(timeseries_bank.keys())
for i in range(len(valid_sym_list)-1):
    for j in range(i+1, len(valid_sym_list)):
        sym_pairs.append((valid_sym_list[i], valid_sym_list[j]))
for pair in sym_pairs:
    sym_pairs_ts[pair] = {"first":timeseries_bank[pair[0]], "second":timeseries_bank[pair[1]]}
    
# sym_paris_ts is the pair timeseries:
# {(sym1, sym2):{'first':[(datetime.date(2010,1,1),price1), (datetime.date(2010,1,2):price2), ...}, "second":[...]}}

In [22]:
def OUCalibration(pairs,start_date,end_date,data):
    """
    pair: list of tuple
    date: datetime.date(YYYY,MM,DD)
    data: sym_pairs_ts
    """
    timestamp=np.array(list(data[pairs[0]]["first"].keys())).flatten()
    dateRange=(timestamp>=start_date) & (timestamp<=end_date)
    ans={}
    deltat=1/252
    for pair in pairs:
        t1=np.log(np.array(list(data[pair]["first"].values())).flatten()[dateRange])
        t2=np.log(np.array(list(data[pair]["second"].values())).flatten()[dateRange])

        t2_constant=sm.add_constant(t2)
        model2 = sm.OLS(t1,t2_constant)
        result2 = model2.fit()
        a1,b1=result2.params
        
        
        Xt=t1-a1-b1*t2
        y=Xt[1:]
        x=Xt[:-1]
        x=sm.add_constant(x)
        model = sm.OLS(y,x)
        results = model.fit()
        a,b=results.params
        std_residual=np.std(results.resid)
        
        theta=-np.log(b)/deltat
        mu=a/(1-b)
        sigma=std_residual*np.sqrt(2*theta/(1-b**2))
        
        df=pd.DataFrame()
        
        ans[pair]={"half_life":np.log(2)/theta*252,"mu":mu,"sigma":sigma/np.sqrt(2*theta),"const":a1,"coef":b1}
        
    return ans

In [12]:
#用这些代码做测试，看看算出来的数值正不正常,CointegrationTest 在toolbox.py里面
listOfPairs=list(sym_pairs_ts.keys())
startDate=datetime.date(2012,10,1)
endDate=datetime.date(2013,10,1)
result1=CointegrationTest(listOfPairs,startDate,endDate,timeseries_bank)
result2=OUCalibration(result1,startDate,endDate,sym_pairs_ts)
print(result2)

dict_keys(['first', 'second'])